In [75]:
#CSV Bibliothek importieren 
import csv
#Pandas importieren und als pd bezeichnen
import pandas as pd
#Numpy importieren und als np bezeichnen 
import numpy as np
#Json Biliothek importieren
import json
#Obere Schranke für die Größe der Daten auf 100,000,000 setzen  
csv.field_size_limit(100000000)
#Maximale Anzahl von Zeilen und Spalten Zeigen 
pd.options.display.max_columns = None
pd.options.display.max_rows = None
#Extraktion der Domäne
import tldextract

***

### Pro_science domains

In [76]:
path = '../data/df_domains_pro_anti_science.tsv'

df_domains_pro_and_anti_science = pd.read_csv(path, sep='\t', error_bad_lines=False)

In [77]:
print('Shape: ', df_domains_pro_and_anti_science.shape)
df_domains_pro_and_anti_science.head()

Shape:  (1395, 4)


,domain,mediabias_proscience,mediabias_questionblesources,mediabias_conspiracy_psydoscience
0,Covid.gov,1.0,0.0,0.0
1,Drugs.com,1.0,0.0,0.0
2,News-Medical.net,1.0,0.0,0.0
3,SciTechDaily.com,1.0,0.0,0.0
4,aaas.org,1.0,0.0,0.0


In [78]:
df_proscience_domains = df_domains_pro_and_anti_science[df_domains_pro_and_anti_science['mediabias_proscience'] == 1]

print('Shape: ', df_proscience_domains.shape)
df_proscience_domains.head()

Shape:  (150, 4)


,domain,mediabias_proscience,mediabias_questionblesources,mediabias_conspiracy_psydoscience
0,Covid.gov,1.0,0.0,0.0
1,Drugs.com,1.0,0.0,0.0
2,News-Medical.net,1.0,0.0,0.0
3,SciTechDaily.com,1.0,0.0,0.0
4,aaas.org,1.0,0.0,0.0


In [79]:
df_proscience_domains.domain.unique().shape[0]

150

In [80]:
list_proscience_domains = df_proscience_domains.domain.tolist()

In [81]:
len(list_proscience_domains)

150

***

## Anti_science

***

In [82]:
path = '../data/df_domains_conspiracy_psydoscience_337.tsv'

df_domains_anti_science = pd.read_csv(path, sep='\t', error_bad_lines=False)

In [83]:
df_domains_anti_science.domain.unique().shape[0]

337

In [84]:
list_antiscience_domains = df_domains_anti_science.domain.tolist()

In [85]:
len(list_antiscience_domains)

337

## Polarisierungen berechnen
### TweetsCov19

---

In [86]:
path_tweetscov19 = '../data/tweetscov19_id_isdeleted_domains.tsv'

df_tweetscov19 = pd.read_csv(path_tweetscov19, sep='\t', error_bad_lines=False, names=['Tweet_Id', 'Username', 'Is_deleted', 'Domains'])

print('Shape :', df_tweetscov19.shape)
df_tweetscov19.head()

Shape : (9409841, 4)


,Tweet_Id,Username,Is_deleted,Domains
0,1178791637348212736,c57d4fb895c5f4a33d2a2b24709f93b5,True,0
1,1178791641601232896,627f7d4ef584ba33a676a4ebcb861cdb,False,catholicnewsagency.com
2,1178791643509612547,7966d5716c1c4f33591b17755184e3d6,False,0
3,1178791644214235137,f17996c036ed4cd981f5448fe00d9ed2,False,0
4,1178791645371875329,32a54e5f6d8a67de1bffd2e689594dc8,False,0


In [87]:
def get_number_of_antiscience_domains(domains):
    domains_list = domains.split()
    score = 0
    for i in range(0, len(domains_list)):
        
        #d = tldextract.extract(domains_list[i])
        #d = d.domain          
        d = domains_list[i]
        
        if d in list_antiscience_domains:
            score -=  1
        
    return score

In [88]:
df_tweetscov19['Domains'] = df_tweetscov19['Domains'].astype(str)

df_tweetscov19['number_of_antisciens_domains'] = df_tweetscov19.apply(lambda row: get_number_of_antiscience_domains(row['Domains']) if (row['Domains'] != '0') else 0, axis=1)

In [89]:
df_tweetscov19.head()

,Tweet_Id,Username,Is_deleted,Domains,number_of_antisciens_domains
0,1178791637348212736,c57d4fb895c5f4a33d2a2b24709f93b5,True,0,0
1,1178791641601232896,627f7d4ef584ba33a676a4ebcb861cdb,False,catholicnewsagency.com,0
2,1178791643509612547,7966d5716c1c4f33591b17755184e3d6,False,0,0
3,1178791644214235137,f17996c036ed4cd981f5448fe00d9ed2,False,0,0
4,1178791645371875329,32a54e5f6d8a67de1bffd2e689594dc8,False,0,0


In [90]:
tweets_with_antiscience_domains = df_tweetscov19[df_tweetscov19['number_of_antisciens_domains'] < 0].shape

print('Number of tweets with at least one antiscience Domain: ', tweets_with_antiscience_domains[0])

Number of tweets with at least one antiscience Domain:  20246


In [91]:
def get_number_of_proscience_domains(domains):
    domains_list = domains.split()
    score = 0
    for i in range(0, len(domains_list)):
        
        #d = tldextract.extract(domains_list[i])
        #d = d.domain
       
            
        d = domains_list[i]
        if d in list_proscience_domains:
            score +=  1
        
    return score

In [92]:
df_tweetscov19['number_of_prosciens_domains'] = df_tweetscov19.apply(lambda row: get_number_of_proscience_domains(row['Domains']) if (row['Domains'] != '0') else 0, axis=1)

In [93]:
df_tweetscov19.head()

,Tweet_Id,Username,Is_deleted,Domains,number_of_antisciens_domains,number_of_prosciens_domains
0,1178791637348212736,c57d4fb895c5f4a33d2a2b24709f93b5,True,0,0,0
1,1178791641601232896,627f7d4ef584ba33a676a4ebcb861cdb,False,catholicnewsagency.com,0,0
2,1178791643509612547,7966d5716c1c4f33591b17755184e3d6,False,0,0,0
3,1178791644214235137,f17996c036ed4cd981f5448fe00d9ed2,False,0,0,0
4,1178791645371875329,32a54e5f6d8a67de1bffd2e689594dc8,False,0,0,0


In [94]:
tweets_with_proscience_domains = df_tweetscov19[df_tweetscov19['number_of_prosciens_domains'] > 0].shape

print('Number of tweets with at least one proscience domain: ', tweets_with_proscience_domains[0])

Number of tweets with at least one proscience domain:  34962


In [95]:

df_tweetscov19_unique_user = df_tweetscov19.groupby('Username')[ 'number_of_antisciens_domains', 'number_of_prosciens_domains'].sum()

<ipython-input-95-5a729d8510a2>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_tweetscov19_unique_user = df_tweetscov19.groupby('Username')[ 'number_of_antisciens_domains', 'number_of_prosciens_domains'].sum()


In [96]:
number_of_unique_useres = df_tweetscov19_unique_user.shape

print('Number of unique users in TweetsCOV19: ', number_of_unique_useres[0])

Number of unique users in TweetsCOV19:  4095876


In [97]:
df_tweetscov19_unique_user[df_tweetscov19_unique_user['number_of_prosciens_domains']>0].head()

,number_of_antisciens_domains,number_of_prosciens_domains
Username,,
0002eb2aff20e181397afe8cdebdd395,0,1
0006a238065c3acb128041e666b58f07,0,2
0008667130fe3776676804ff59e8ce3b,0,3
000d4b29789dbf222f58a0b29a63b0e1,0,1
000e3562755686866515f367bb4d457b,0,1


In [98]:
df_tweetscov19_unique_user['Sum_polarized_domains'] = df_tweetscov19_unique_user.apply(lambda row:  (row['number_of_prosciens_domains'] - row['number_of_antisciens_domains']), axis=1)

In [99]:
df_tweetscov19_unique_user[df_tweetscov19_unique_user['number_of_antisciens_domains']<0].head()

,number_of_antisciens_domains,number_of_prosciens_domains,Sum_polarized_domains
Username,,,
000bcb01b05c0f275601b470ab115a06,-1,0,1
0018a0a27af297572e872e1f6ba30745,-1,0,1
001e4d1b649f5e17fbe9cd0799a9ad5f,-1,0,1
0023797612d9ad76ffd7024c324c499c,-1,0,1
0032f128aa4d46bebef52935243c2a46,-1,0,1


In [100]:
df_tweetscov19_unique_user['normalised_polarisation'] = df_tweetscov19_unique_user.apply(lambda row: (row['number_of_antisciens_domains'] + row['number_of_prosciens_domains']) / row['Sum_polarized_domains'] if row['Sum_polarized_domains'] != 0 else 0, axis=1)

In [101]:
df_tweetscov19_unique_user[df_tweetscov19_unique_user['number_of_antisciens_domains']<-2].head()

,number_of_antisciens_domains,number_of_prosciens_domains,Sum_polarized_domains,normalised_polarisation
Username,,,,
0096f2ad8e3e1a5d9b9e6c55864d1ca4,-3,0,3,-1.0
00fb1a18f16ad2432da347397f6a6a6b,-5,0,5,-1.0
016a56fb48e42132723b50efa27b19fd,-6,0,6,-1.0
01baae45b1a3710bf078f4e14c409ea0,-4,0,4,-1.0
01cf9cf2a2c738f5f7c5cab479c4322f,-8,0,8,-1.0


In [102]:
users_with_with_atleast_one_polarized_domain = df_tweetscov19_unique_user[df_tweetscov19_unique_user['Sum_polarized_domains']>0]

print('Number of users with with at least one polarized domain: ',users_with_with_atleast_one_polarized_domain.shape[0])

Number of users with with at least one polarized domain:  25491


In [103]:
# users_with_with_atleast_one_polarized_domain[users_with_with_atleast_one_polarized_domain['normalised_polarisation'] == 0].head()

In [104]:
#df_tweetscov19_unique_user_without_zero_polarisation = df_tweetscov19_unique_user[df_tweetscov19_unique_user['normalised_polarisation'] != 0]

In [105]:
#number_of_science_polarized_users = df_tweetscov19_unique_user_without_zero_polarisation.shape

#print('Number of science polarized users in TWeetsCOV19: ', number_of_science_polarized_users[0])

In [106]:
number_of_proscience_polarized_users = users_with_with_atleast_one_polarized_domain[users_with_with_atleast_one_polarized_domain['normalised_polarisation'] > 0].shape

print('Number of proscience polarized users in TWeetsCOV19: ', number_of_proscience_polarized_users[0])

Number of proscience polarized users in TWeetsCOV19:  19216


In [107]:
number_of_antiscience_polarized_users = users_with_with_atleast_one_polarized_domain[users_with_with_atleast_one_polarized_domain['normalised_polarisation'] < 0].shape

print('Number of antiscience polarized users in TWeetsCOV19: ', number_of_antiscience_polarized_users[0])

Number of antiscience polarized users in TWeetsCOV19:  6024


****

In [108]:
users_with_with_atleast_one_polarized_domain = users_with_with_atleast_one_polarized_domain.reset_index()
users_with_with_atleast_one_polarized_domain.head()

,Username,number_of_antisciens_domains,number_of_prosciens_domains,Sum_polarized_domains,normalised_polarisation
0,0002eb2aff20e181397afe8cdebdd395,0,1,1,1.0
1,0006a238065c3acb128041e666b58f07,0,2,2,1.0
2,0008667130fe3776676804ff59e8ce3b,0,3,3,1.0
3,000bcb01b05c0f275601b470ab115a06,-1,0,1,-1.0
4,000d4b29789dbf222f58a0b29a63b0e1,0,1,1,1.0


***

***

In [54]:
df_tweetscov19 = df_tweetscov19[['Tweet_Id', 'number_of_antisciens_domains', 'number_of_prosciens_domains']]


path = '../data/tweetscov19_tweets_science_polarized_v3.tsv'

with open(path, 'w', newline='', encoding="utf-8") as out_file:
    tsv_writer = csv.writer(out_file, delimiter="\t")
    for idx, row in df_tweetscov19.iterrows():
        tsv_writer.writerow(row.values.tolist())

***

In [109]:
pfad_tsv3 = '../data/tweetscov19_science_polarized_users_with_atleast_one_polarized_domain_v3.tsv'

with open(pfad_tsv3, 'w', newline='', encoding="utf-8") as out_file:
    tsv_writer = csv.writer(out_file, delimiter="\t")
    for idx, row in users_with_with_atleast_one_polarized_domain.iterrows():
        tsv_writer.writerow(row.values.tolist())

***

***

## Polarisierungen berechnen
### TweetsKB

***

In [110]:
pfad_tsv = '../data/tweetskb_id_isdeleted_domains.tsv'

df_tweetskb = pd.read_csv(pfad_tsv, sep='\t', error_bad_lines=False, names=['Tweet_Id', 'Username', 'Is_deleted', 'Domains'])

print('Shape :', df_tweetskb.shape)
df_tweetskb.head()

Shape : (9409841, 4)


,Tweet_Id,Username,Is_deleted,Domains
0,1178791636039589889,DCComics,False,dcuniverseinfinite.com
1,1178791641265639424,seanberdyz,True,carrd.co
2,1178791640573579264,AnArtistAtBirth,True,0
3,1178791644797255680,lagosboygang,True,0
4,1178791648056217600,Phenom_Hoops,False,phenomhoopreport.com


In [111]:
df_tweetskb['Domains'] = df_tweetskb['Domains'].astype(str)
df_tweetskb['number_of_antisciens_domains'] = df_tweetskb.apply(lambda row: get_number_of_antiscience_domains(row['Domains']) if (row['Domains'] != '0') else 0, axis=1)

In [112]:
tweets_with_antiscience_domains = df_tweetskb[df_tweetskb['number_of_antisciens_domains'] < 0].shape

print('Number of tweets with at least one antiscience Domain: ', tweets_with_antiscience_domains[0])

Number of tweets with at least one antiscience Domain:  4392


In [113]:
df_tweetskb['number_of_prosciens_domains'] = df_tweetskb.apply(lambda row: get_number_of_proscience_domains(row['Domains']) if (row['Domains'] != '0') else 0, axis=1)

In [114]:
tweets_with_proscience_domains = df_tweetskb[df_tweetskb['number_of_prosciens_domains'] > 0].shape

print('Number of tweets with at least one antiscience Domain: ', tweets_with_proscience_domains[0])

Number of tweets with at least one antiscience Domain:  6599


In [115]:

df_tweetskb_unique_user = df_tweetskb.groupby('Username')[ 'number_of_antisciens_domains', 'number_of_prosciens_domains'].sum()

<ipython-input-115-532f45cc846d>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_tweetskb_unique_user = df_tweetskb.groupby('Username')[ 'number_of_antisciens_domains', 'number_of_prosciens_domains'].sum()


In [116]:
unique_users = df_tweetskb_unique_user.shape

print('Number of unique users in TweetsKB: ', unique_users[0])

Number of unique users in TweetsKB:  5208425


In [117]:
df_tweetskb_unique_user.head()

,number_of_antisciens_domains,number_of_prosciens_domains
Username,,
00000000,0,0
000000000yt,0,0
00000000martin,0,0
000000911a,0,0
000000_127,0,0


In [118]:
df_tweetskb_unique_user['Sum_polarized_domains'] = df_tweetskb_unique_user.apply(lambda row: (row['number_of_prosciens_domains'] - row['number_of_antisciens_domains']), axis=1)

In [119]:
df_tweetskb_unique_user[df_tweetskb_unique_user['number_of_antisciens_domains']<-2].head()

,number_of_antisciens_domains,number_of_prosciens_domains,Sum_polarized_domains
Username,,,
1foolon1hill,-3,0,3
21WIRE,-11,0,11
7SealsOfTheEnd,-4,0,4
ADDitudeMag,-7,0,7
AIIAmericanGirI,-6,0,6


In [120]:
df_tweetskb_unique_user['normalised_polarisation'] = df_tweetskb_unique_user.apply(lambda row: (row['number_of_antisciens_domains'] + row['number_of_prosciens_domains']) / row['Sum_polarized_domains'] if row['Sum_polarized_domains'] != 0 else 0, axis=1)

In [121]:
df_tweetskb_unique_users_with_atleast_one_polarized_domain= df_tweetskb_unique_user[df_tweetskb_unique_user['Sum_polarized_domains'] >0]

print('df tweetskb unique users with atleast one polarized domain: ', df_tweetskb_unique_users_with_atleast_one_polarized_domain.shape[0])

df tweetskb unique users with atleast one polarized domain:  7004


In [122]:
df_tweetskb_unique_users_with_atleast_one_polarized_domain[df_tweetskb_unique_users_with_atleast_one_polarized_domain['normalised_polarisation']==0].head()

,number_of_antisciens_domains,number_of_prosciens_domains,Sum_polarized_domains,normalised_polarisation
Username,,,,
1foreverseeking,-1,1,2,0.0
AgentApplebutt3,-1,1,2,0.0
BeachMilk,-1,1,2,0.0
BuzzPatterson,-1,1,2,0.0
COsweda,-1,1,2,0.0


In [123]:
#df_tweetskb_unique_user_without_zero_polarisation = df_tweetskb_unique_user[df_tweetskb_unique_user['normalised_polarisation'] != 0]

In [124]:
#number_of_science_polarized_users = df_tweetskb_unique_user_without_zero_polarisation.shape

#print('Number of science polarized users in TWeetsKB: ', number_of_science_polarized_users[0])

In [125]:
number_of_proscience_polarized_users = df_tweetskb_unique_users_with_atleast_one_polarized_domain[df_tweetskb_unique_users_with_atleast_one_polarized_domain['normalised_polarisation'] > 0].shape

print('Number of proscience polarized users in TWeetsKB: ', number_of_proscience_polarized_users[0])

Number of proscience polarized users in TWeetsKB:  4855


In [126]:
number_of_antiscience_polarized_users = df_tweetskb_unique_users_with_atleast_one_polarized_domain[df_tweetskb_unique_users_with_atleast_one_polarized_domain['normalised_polarisation'] < 0].shape

print('Number of antiscience polarized users in TWeetsKB: ', number_of_antiscience_polarized_users[0])

Number of antiscience polarized users in TWeetsKB:  2118


In [127]:
df_tweetskb_unique_users_with_atleast_one_polarized_domain = df_tweetskb_unique_users_with_atleast_one_polarized_domain.reset_index()
df_tweetskb_unique_users_with_atleast_one_polarized_domain.head()

,Username,number_of_antisciens_domains,number_of_prosciens_domains,Sum_polarized_domains,normalised_polarisation
0,01sth02,0,1,1,1.0
1,0NoMyProfile,0,1,1,1.0
2,0_TruthHurts_0,0,1,1,1.0
3,0dayDB,0,1,1,1.0
4,101stmonk3y,-1,0,1,-1.0


***

***

In [73]:
df_tweetskb = df_tweetskb[['Tweet_Id', 'number_of_antisciens_domains', 'number_of_prosciens_domains']]

path = '../data/tweetskb_tweets_science_polarized_v3.tsv'

with open(path, 'w', newline='', encoding="utf-8") as out_file:
    tsv_writer = csv.writer(out_file, delimiter="\t")
    for idx, row in df_tweetskb.iterrows():
        tsv_writer.writerow(row.values.tolist())

***

***

In [128]:
pfad_tsv3 = '../data/tweetskb_science_polarized_users_with_atleast_one_polarized_domain_v3.tsv'

with open(pfad_tsv3, 'w', newline='', encoding="utf-8") as out_file:
    tsv_writer = csv.writer(out_file, delimiter="\t")
    for idx, row in df_tweetskb_unique_users_with_atleast_one_polarized_domain.iterrows():
        tsv_writer.writerow(row.values.tolist())